In [ ]:
import numpy as np
import evaluate
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

dataset = load_dataset("imdb")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

encoded_dataset = dataset.map(tokenize, batched=True)
encoded_dataset = encoded_dataset.remove_columns(["text"]).rename_column("label", "labels")
encoded_dataset.set_format("torch")


model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    return metric.compute(predictions=np.argmax(logits, axis=-1), references=labels)


args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    report_to="none",  # <--- THIS LINE STOPS THE LOGIN PROMPT
    num_train_epochs=1, # Reduced to 1 for speed (3 takes a long time on CPU)
    per_device_train_batch_size=8, # 8 is safer for smaller GPUs/CPUs
)


trainer = Trainer(
    model=model,
    args=args,
    train_dataset=encoded_dataset["train"].shuffle(seed=42).select(range(1000)), # Small subset for testing
    eval_dataset=encoded_dataset["test"].shuffle(seed=42).select(range(100)),
    compute_metrics=compute_metrics,
)

trainer.train()

print(trainer.evaluate())

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [2]:
!pip install evaluate
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.9 MB/s eta 0:00:00
